In [1]:
from itertools import chain
import pandas as pd

csv = 'files/bbs.csv'
# Load the CSV file (and 
#  the unnecessary columns).
df = pd.read_csv(csv)
df_10 = df[df['Dia'] == 10] 
df_10["Qty"] = df_10["Qty"] // 24
df_10.loc[:,'Qty'] = df_10.loc[:,'Qty'].astype(int)
df_10['new'] = df_10.apply(lambda row: [row['Length']] * row['Qty'], axis=1)
weights = list(chain.from_iterable(df_10['new']))
len(weights)

/var/folders/_g/bjx3wyj92f9gjghwwr9y32wr0000gp/T/ipykernel_6540/514771618.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10["Qty"] = df_10["Qty"] // 24
/var/folders/_g/bjx3wyj92f9gjghwwr9y32wr0000gp/T/ipykernel_6540/514771618.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['new'] = df_10.apply(lambda row: [row['Length']] * row['Qty'], axis=1)


25

In [5]:
from ortools.linear_solver import pywraplp


def create_data_model(weights):
    """Create the data for the example."""
    data = {}
    data['weights'] = weights
    data['items'] = list(range(len(weights)))
    data['bins'] = data['items']
    data['bin_capacity'] = 11700
    return data

def main():
    data = create_data_model(weights)

    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    #solver.set_time_limit(60)

    if not solver:
        return

    # Variables
    # x[i, j] = 1 if item i is packed in bin j.
    x = {}
    for i in data['items']:
        for j in data['bins']:
            x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

    # y[j] = 1 if bin j is used.
    y = {}
    for j in data['bins']:
        y[j] = solver.IntVar(0, 1, 'y[%i]' % j)

    # Constraints
    # Each item must be in exactly one bin.
    for i in data['items']:
        solver.Add(sum(x[i, j] for j in data['bins']) == 1)

    # The amount packed in each bin cannot exceed its capacity.
    for j in data['bins']:
        solver.Add(
            sum(x[(i, j)] * data['weights'][i] for i in data['items']) <= y[j] * data['bin_capacity'])

    # Objective: minimize the number of bins used.
    solver.Minimize(solver.Sum([y[j] for j in data['bins']]))

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        num_bins = 0
        for j in data['bins']:
            if y[j].solution_value() == 1:
                bin_items = []
                bin_weight = 0
                for i in data['items']:
                    if x[i, j].solution_value() > 0:
                        bin_items.append(i)
                        bin_weight += data['weights'][i]
                if bin_items:
                    num_bins += 1
                    print('Bin number', j)
                    print('  Items packed:', bin_items)
                    print('  Total weight:', bin_weight)
                    print()
        print()
        print('Number of bins used:', num_bins)
        print('Time = ', solver.WallTime(), ' milliseconds')
    else:
        print('The problem does not have an optimal solution.')


if __name__ == '__main__':
    main()

Bin number 0
  Items packed: [1, 6, 9, 14]
  Total weight: 11440

Bin number 1
  Items packed: [18, 19, 24]
  Total weight: 11490

Bin number 5
  Items packed: [2, 5, 7, 10]
  Total weight: 11440

Bin number 10
  Items packed: [15, 21, 23]
  Total weight: 11490

Bin number 17
  Items packed: [3, 11, 12, 16]
  Total weight: 11440

Bin number 22
  Items packed: [13, 20, 22]
  Total weight: 11490

Bin number 24
  Items packed: [0, 4, 8, 17]
  Total weight: 11210


Number of bins used: 7
Time =  15321  milliseconds
